<a href="https://colab.research.google.com/github/HyeJin816/ESAA_22/blob/main/0328_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

In [4]:
X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_test_val, y_train, y_test_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [6]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

svm_clf = SVC(random_state=42, probability=True)

mlp_clf = MLPClassifier(random_state=42)

In [7]:
random_forest_clf.fit(X_train, y_train)
extra_trees_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)

MLPClassifier(random_state=42)

1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [8]:
from sklearn.ensemble import VotingClassifier

In [ ]:
voting_clf = VotingClassifier(
    estimators = [('rnd', random_forest_clf),
                  ('ext', extra_trees_clf),
                  ('svm', svm_clf),
                  ('mlp', mlp_clf)],
    voting='hard')

voting_clf.fit(X_train, y_train)

In [ ]:
voting_pred = voting_clf.predict(X_test)

1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [ ]:
voting_clf2 = VotingClassifier(
    estimators = [('rnd', random_forest_clf),
                  ('ext', extra_trees_clf),
                  ('mlp', mlp_clf)],
    voting='hard')

voting_clf2.fit(X_train, y_train)

In [ ]:
voting_pred2 = voting_clf2.predict(X_test)

In [ ]:
# SVM 유무에 따른 성능 비교

from sklearn.metrics import accuracy_score

print('VotingClassifier', accuracy_score(y_test, voting_pred))
print('SVM 제거', accuracy_score(y_test, voting_pred2))

1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [ ]:
voting_clf3 = VotingClassifier(
    estimators = [('rnd', random_forest_clf),
                  ('ext', extra_trees_clf),
 #                 ('svm', svm_clf),
                  ('mlp', mlp_clf)],
    voting='soft')

voting_clf3.fit(X_train, y_train)

In [ ]:
voting_pred3 = voting_clf3.predict(X_test)

print('간접투표분류기', accuracy_score(y_test, voting_pred3))

#voting_pred3.score(X_val,y_val)